In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Bild einlesen
img = cv2.imread('Utils/links-muster-3.png')

# Bild in Graustufen umwandeln
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Kanten mit dem Canny-Algorithmus detektieren
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Kanten anzeigen
plt.figure(figsize=(10, 6))
plt.subplot(121), plt.imshow(gray, cmap='gray')
plt.title('Graustufenbild'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(edges, cmap='gray')
plt.title('Kantendetektion'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Bild einlesen
img = cv2.imread('Utils/links-muster-3.png')
# Bild in Graustufen umwandeln
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Prewitt-Filter definieren
prewitt_kernel_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
prewitt_kernel_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])

# Filter anwenden
grad_x = cv2.filter2D(gray, cv2.CV_64F, prewitt_kernel_x)
grad_y = cv2.filter2D(gray, cv2.CV_64F, prewitt_kernel_y)

# Gradientenbilder kombinieren
magnitude = np.sqrt(grad_x**2 + grad_y**2)
magnitude = cv2.convertScaleAbs(magnitude)

# Ergebnisse anzeigen
plt.figure(figsize=(12, 6))
plt.subplot(131), plt.imshow(gray, cmap='gray')
plt.title('Graustufenbild'), plt.xticks([]), plt.yticks([])
plt.subplot(132), plt.imshow(grad_x, cmap='gray')
plt.title('Prewitt X'), plt.xticks([]), plt.yticks([])
plt.subplot(133), plt.imshow(grad_y, cmap='gray')
plt.title('Prewitt Y'), plt.xticks([]), plt.yticks([])
plt.show()

plt.figure(figsize=(6, 6))
plt.imshow(magnitude, cmap='gray')
plt.title('Prewitt-Kantendetektion'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Bild einlesen
img = cv2.imread('Utils/links-muster-3.png')

# Überprüfen, ob das Bild korrekt eingelesen wurde
if img is None:
    raise FileNotFoundError("Das Bild konnte nicht geladen werden. Bitte überprüfen Sie den Pfad.")

# Bild in Graustufen umwandeln
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Prewitt-Filter für horizontale Richtung definieren
prewitt_kernel_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])

# Horizontalen Prewitt-Filter anwenden
grad_x = cv2.filter2D(gray, cv2.CV_64F, prewitt_kernel_x)

# Betrag der horizontalen Gradienten berechnen
abs_grad_x = np.abs(grad_x)

# Optional: Ausgabe als Bild zur Überprüfung
abs_grad_x_img = cv2.convertScaleAbs(abs_grad_x)
plt.figure(figsize=(12, 6))
plt.subplot(121), plt.imshow(gray, cmap='gray')
plt.title('Graustufenbild'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(abs_grad_x_img, cmap='gray')
plt.title('Horizontale Gradienten (Prewitt X)'), plt.xticks([]), plt.yticks([])


# Horizontale Gradienten als numerische Werte ausgeben
print("Horizontale Gradienten (Prewitt X):")
print(abs_grad_x)

# Beispiel: Durchschnittliche Änderung pro Zeile
average_change_per_row = np.mean(abs_grad_x, axis=1)
print("\nDurchschnittliche horizontale Änderung pro Zeile:")
print(average_change_per_row)

# Beispiel: Durchschnittliche Änderung pro Spalte
average_change_per_column = np.mean(abs_grad_x, axis=0)
print("\nDurchschnittliche horizontale Änderung pro Spalte:")
print(average_change_per_column)
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Laden der aufgenommenen Bilder (linke und rechte Ansicht)
imgL = cv2.imread('Utils/lasertestv2-links.png', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('Utils/lasertestv2-rechts.png', cv2.IMREAD_GRAYSCALE)

# Laden des Gittermusters (Pattern), das auf die Szene projiziert wurde
pattern = cv2.imread('Utils/lasertestv2-links2.png', cv2.IMREAD_GRAYSCALE)

def undistort_pattern(img, pattern):
    # Annahme: Das Gittermuster wurde nur in der horizontalen oder vertikalen Richtung verzerrt.
    
    # Find corners in pattern
    ret, corners_pattern = cv2.findChessboardCorners(pattern, (9, 6), None)
    
    if not ret:
        raise ValueError("Gittermuster konnte nicht gefunden werden.")
    
    # Find corners in input image
    ret, corners_img = cv2.findChessboardCorners(img, (9, 6), None)
    
    if not ret:
        raise ValueError("Gittermuster im Bild konnte nicht gefunden werden.")

    return corners_pattern, corners_img

def compute_depth_map(corners_pattern, corners_img, baseline_cm, focal_length_mm):
    baseline = baseline_cm / 100.0
    focal_length_px = (focal_length_mm / 32.0) * 1000

    depth_map = np.zeros((corners_img.shape[0], 1), dtype=np.float32)
    
    disparities = corners_pattern[:,:,0] - corners_img[:,:,0]  # Nur x-Koordinate
    for i in range(len(disparities)):
        disparity = disparities[i]
        if disparity > 0:
            depth = (focal_length_px * baseline) / disparity
            depth_map[i] = depth

    return depth_map

# Definiere die Baseline (Abstand zwischen den Kameras in cm) und die Brennweite (in mm)
baseline_cm = 7
focal_length_mm = 32

# Unverzerrte und verzerrte Gitterpunkte finden
corners_pattern, corners_imgL = undistort_pattern(imgL, pattern)
corners_pattern, corners_imgR = undistort_pattern(imgR, pattern)

# Berechnung der Tiefenkarte für linkes Bild
depth_map_L = compute_depth_map(corners_pattern, corners_imgL, baseline_cm, focal_length_mm)
depth_map_R = compute_depth_map(corners_pattern, corners_imgR, baseline_cm, focal_length_mm)

# Zeigen der Tiefenkarte
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(depth_map_L, cmap='plasma')
plt.colorbar(label='Tiefe (Meter)')
plt.title('Tiefenkarte Links')

plt.subplot(1, 2, 2)
plt.imshow(depth_map_R, cmap='plasma')
plt.colorbar(label='Tiefe (Meter)')
plt.title('Tiefenkarte Rechts')

plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Einlesen der Bilder
imgL = cv2.imread('Utils/lasertestv2-links.png', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('Utils/lasertestv2-rechts.png', cv2.IMREAD_GRAYSCALE)
pattern = cv2.imread('Utils/lasertestv2-links2.png', cv2.IMREAD_GRAYSCALE)

# Funktion zur Erkennung und Berechnung der Tiefenkarte basierend auf Gitterlinienverzerrung
def create_depth_map_from_pattern(img, pattern, baseline_cm, focal_length_mm):
    # Find the edges of the pattern
    edges_pattern = cv2.Canny(pattern, 50, 150)
    edges_img = cv2.Canny(img, 50, 150)

    # Hough Line Transform to detect lines in both images
    lines_pattern = cv2.HoughLines(edges_pattern, 1, np.pi / 180, 200)
    lines_img = cv2.HoughLines(edges_img, 1, np.pi / 180, 200)

    # Calculate the disparity of lines between the pattern and the image
    baseline = baseline_cm / 100.0
    focal_length_px = (focal_length_mm / 32.0) * 1000

    depth_map = np.zeros_like(img, dtype=np.float32)

    for pattern_line in lines_pattern:
        rho_pat, theta_pat = pattern_line[0]
        for img_line in lines_img:
            rho_img, theta_img = img_line[0]

            # Calculate the disparity (simplified as difference in rho, assuming horizontal lines)
            disparity = np.abs(rho_pat - rho_img)
            if disparity > 0:
                depth = (focal_length_px * baseline) / disparity
                if 0 <= int(rho_img) < depth_map.shape[0]:
                    depth_map[int(rho_img)] = depth

    # Normalisieren und Umwandeln zur Visualisierung
    depth_map = cv2.normalize(depth_map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    depth_map = np.uint8(depth_map)

    return depth_map

# Definiere die Baseline (Abstand zwischen den Kameras in cm) und die Brennweite (in mm)
baseline_cm = 7
focal_length_mm = 32

# Tiefenkarte aus dem Gittermuster erstellen
depth_map = create_depth_map_from_pattern(imgL, pattern, baseline_cm, focal_length_mm)

# Tiefenkarte anzeigen
plt.figure(figsize=(10, 5))
plt.imshow(depth_map, cmap='plasma')
plt.colorbar(label='Tiefe (Meter)')
plt.title('Tiefenkarte aus projiziertem Gittermuster')
plt.show()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Lade die Stereo-Bilder
imgL = cv2.imread('Utils/lasertestv2-links.png', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('Utils/lasertestv2-rechts.png', cv2.IMREAD_GRAYSCALE)

# Initiere den Stereo-SBM-Algorithmus (Stereo Block Matching)
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)

# Berechne die Disparität (Differenz der Pixelkoordinaten zwischen den beiden Bildern)
disparity = stereo.compute(imgL, imgR)

# Normiere die Disparität zur Visualisierung
disparity_normalized = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX).astype(np.uint8)

# Erstelle eine Farbkodierte Tiefenkarte (optional)
depth_colormap = cv2.applyColorMap(disparity_normalized, cv2.COLORMAP_JET)

# Zeige die Ergebnisse
plt.subplot(221), plt.imshow(imgL, cmap='gray'), plt.title('Linkes Bild')
plt.subplot(222), plt.imshow(imgR, cmap='gray'), plt.title('Rechtes Bild')
plt.subplot(223), plt.imshow(disparity, cmap='plasma'), plt.title('Disparitätskarte')
plt.subplot(224), plt.imshow(depth_colormap), plt.title('Farbkodierte Tiefenkarte')

plt.show()

# Speichere die Ergebnisse
cv2.imwrite('disparity_map.png', disparity)
cv2.imwrite('depth_colormap.png', depth_colormap)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_lines(img):
    """Detect lines using the Hough Transformation."""
    edges = cv2.Canny(img, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=80, minLineLength=50, maxLineGap=10)
    return lines

def filter_lines(lines, img_shape):
    """Filter and sort lines as left and right"""
    left_lines = []
    right_lines = []
    
    center_x = img_shape[1] // 2
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x1 < center_x and x2 < center_x:
            left_lines.append(line[0])
        elif x1 > center_x and x2 > center_x:
            right_lines.append(line[0])
    
    # Sort lines by y1 position
    left_lines = sorted(left_lines, key=lambda x: min(x[1], x[3]))
    right_lines = sorted(right_lines, key=lambda x: min(x[1], x[3]))
    
    return left_lines, right_lines

def plot_lines(img, lines, title):
    """Plot lines on an image."""
    img_with_lines = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for x1, y1, x2, y2 in lines:
        cv2.line(img_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(img_with_lines)
    plt.title(title)
    plt.show()

def estimate_depth_from_lines(left_lines, right_lines, focal_length, baseline):
    """Estimate depth from line displacement differences"""
    disparities = []
    depths = []
    
    for (l1, r1), (l2, r2) in zip(left_lines, right_lines):
        disparity = np.abs(l1[0] - r1[0])
        disparities.append(disparity)
        z = (focal_length * baseline) / disparity if disparity != 0 else float('inf')
        depths.append(z)
    
    return depths

# Load the stereo images
imgL = cv2.imread('Utils/lasertestv2-links.png', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('Utils/lasertestv2-rechts.png', cv2.IMREAD_GRAYSCALE)

# Detect lines using Hough Transform
linesL = detect_lines(imgL)
linesR = detect_lines(imgR)

# Filter and sort lines into left and right
left_linesL, right_linesL = filter_lines(linesL, imgL.shape)
left_linesR, right_linesR = filter_lines(linesR, imgR.shape)

# Using only the first lines from left and right
left_lineL = left_linesL[0]
right_lineL = right_linesL[0]
left_lineR = left_linesR[0]
right_lineR = right_linesR[0]

# Plot lines for visualization
plot_lines(imgL, [left_lineL, right_lineL], "Linkes Bild with Reference Lines")
plot_lines(imgR, [left_lineR, right_lineR], "Rechtes Bild with Reference Lines")

# Calculate depths based on first reference and subsequent lines (use more lines if present)
focal_length = 3200  # This should be the focal length in pixels
baseline = 50  # This is the distance between the two camera centers

depths = estimate_depth_from_lines([left_lineL, right_lineL], [left_lineR, right_lineR], focal_length, baseline)

print(f"Estimated Depths: {depths}")

# Generate an image showing the estimated depth for each line
depth_image = np.zeros_like(imgL, dtype=np.uint8)
depth_values = []

for depth, (x1, y1, x2, y2) in zip(depths, [left_lineL, right_lineL]):
    depth_values.append(depth)
    cv2.line(depth_image, (x1[0], y1), (x2[0], y2), int(depth / 10), 2)  # Assume depths scaled for visualization

plt.figure(figsize=(10, 8))
plt.imshow(depth_image, cmap='inferno')
plt.title("Tiefenkarte basierend auf Linienverzerrung")
plt.colorbar(label='Tiefe (cm)')
plt.show()

# Display the depth values calculated
for i, depth in enumerate(depth_values, start=1):
    print(f"Tiefe der Linie {i}: {depth:.2f} cm")